In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from pathlib import Path

pd.set_option("display.max_columns", 500)

## Creating Top10 HCPCS CMS PUF file

In [2]:
CY2017 = pd.read_csv("./data/Medicare_Provider_Util_Payment_PUF_CY2017.txt", sep="\t",header=0,skiprows=[1])

/Users/rad/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df = CY2017.copy()

In [4]:
df.columns

Index(['npi', 'nppes_provider_last_org_name', 'nppes_provider_first_name',
       'nppes_provider_mi', 'nppes_credentials', 'nppes_provider_gender',
       'nppes_entity_code', 'nppes_provider_street1', 'nppes_provider_street2',
       'nppes_provider_city', 'nppes_provider_zip', 'nppes_provider_state',
       'nppes_provider_country', 'provider_type',
       'medicare_participation_indicator', 'place_of_service', 'hcpcs_code',
       'hcpcs_description', 'hcpcs_drug_indicator', 'line_srvc_cnt',
       'bene_unique_cnt', 'bene_day_srvc_cnt', 'average_Medicare_allowed_amt',
       'average_submitted_chrg_amt', 'average_Medicare_payment_amt',
       'average_Medicare_standard_amt'],
      dtype='object')

In [5]:
df['Total_submitted_chrg_amt'] = df['line_srvc_cnt'] * df['average_submitted_chrg_amt']
df['Total_Medicare_payment_amt'] = df['line_srvc_cnt'] * df['average_Medicare_payment_amt']

In [6]:
df.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,nppes_provider_zip,nppes_provider_state,nppes_provider_country,provider_type,medicare_participation_indicator,place_of_service,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,Total_Medicare_payment_amt
0,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99217,Hospital observation care discharge,N,100.0,96,100,73.398800,325.780000,56.827200,57.492400,32578.0,5682.72
1,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99218,Hospital observation care typically 30 minutes,N,26.0,25,26,100.080000,449.000000,78.460000,79.306154,11674.0,2039.96
2,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99219,Hospital observation care typically 50 minutes,N,52.0,51,52,136.380000,614.000000,102.807692,103.895385,31928.0,5346.00
3,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99220,Hospital observation care typically 70 minutes...,N,59.0,59,59,190.363729,755.932203,141.293559,142.865763,44600.0,8336.32
4,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99221,"Initial hospital inpatient care, typically 30 ...",N,16.0,16,16,101.680000,462.812500,79.710000,80.750000,7405.0,1275.36


In [7]:
Total_submitted = df.Total_submitted_chrg_amt.sum()

In [8]:
#per-row (per HCPS not grouped) --(wrong code)
#top = df.sort_values(by='Total_submitted_chrg_amt', ascending=False).hcpcs_code.unique() 
#x = df[df.hcpcs_code.isin(top[0:10])]
#x.shape

In [9]:
#x.hcpcs_code.value_counts()

In [10]:
# sort by top 10 hcpcs:
top_hcpcs = df.groupby(['hcpcs_code'])['Total_submitted_chrg_amt'].sum().sort_values(ascending=False)
top10_hcpcs = top_hcpcs[0:10].index
top10_hcpcs

Index(['99214', '99213', '99285', '66984', '99232', '99233', 'A0427', '99223',
       '99291', '99284'],
      dtype='object', name='hcpcs_code')

In [11]:
df_top10 = df[df.hcpcs_code.isin(top10_hcpcs)]
df_top10.shape

(1520985, 28)

In [12]:
df_top10.hcpcs_code.value_counts() #number of submitted charges
# bene_unique_cnt / total_bene_unique_cnt
# unique_npi / total_unique_npi

99213    457843
99214    428268
99232    175336
99223    133368
99233    120664
99291     62680
99284     60805
99285     57220
66984     17613
A0427      7188
Name: hcpcs_code, dtype: int64

In [13]:
pd.DataFrame({'Total_submitted_sum': top_hcpcs[1:10],\
              'Total_submitted_percent': ((top_hcpcs[0:10] / Total_submitted) * 100)})

,Total_submitted_sum,Total_submitted_percent
66984,8.510616e+09,2.695674
99213,1.287074e+10,4.076711
99214,NaN,6.438335
99223,4.681324e+09,1.482774
99232,7.573940e+09,2.398989
99233,5.667375e+09,1.795099
99284,3.748309e+09,1.187249
99285,1.244005e+10,3.940294
99291,4.499186e+09,1.425083
A0427,5.432417e+09,1.720677


In [14]:
top_hcpcs[1:10]

hcpcs_code
99213    1.287074e+10
99285    1.244005e+10
66984    8.510616e+09
99232    7.573940e+09
99233    5.667375e+09
A0427    5.432417e+09
99223    4.681324e+09
99291    4.499186e+09
99284    3.748309e+09
Name: Total_submitted_chrg_amt, dtype: float64

In [15]:
top_hcpcs[0:10] / Total_submitted * 100

hcpcs_code
99214    6.438335
99213    4.076711
99285    3.940294
66984    2.695674
99232    2.398989
99233    1.795099
A0427    1.720677
99223    1.482774
99291    1.425083
99284    1.187249
Name: Total_submitted_chrg_amt, dtype: float64

In [16]:
df_top10.groupby('hcpcs_code')['npi'].count()

hcpcs_code
66984     17613
99213    457843
99214    428268
99223    133368
99232    175336
99233    120664
99284     60805
99285     57220
99291     62680
A0427      7188
Name: npi, dtype: int64

In [17]:
df_top10.shape

(1520985, 28)

In [18]:
df_top10.hcpcs_code.value_counts()

99213    457843
99214    428268
99232    175336
99223    133368
99233    120664
99291     62680
99284     60805
99285     57220
66984     17613
A0427      7188
Name: hcpcs_code, dtype: int64

In [19]:
#df_top10.to_csv(Path("data","top10_hcpcs.csv")) #create a new csv file

In [20]:
#sort via highest avg Total_average_submitted_charge_amount
#pick top 10 
#find proportion for the top 10
# drop all columns not in top 10

# LEIE and TOP-10 HCPCS CleanUP

In [21]:
#LEIE
LEIE = pd.read_csv('data/LEIE_08_2019.csv')

/Users/rad/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [22]:
#Top10-HCPCS
df = pd.read_csv('data/top10_hcpcs.csv', 
                 dtype={'hcpcs_code':object},
                 index_col=0)

/Users/rad/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/rad/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [23]:
df.shape

(1520985, 28)

In [24]:
df.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,nppes_provider_zip,nppes_provider_state,nppes_provider_country,provider_type,medicare_participation_indicator,place_of_service,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,Total_Medicare_payment_amt
6,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99223,"Initial hospital inpatient care, typically 70 ...",N,114.0,110,114,206.061579,923.991228,160.377193,160.026316,105335.000000,18283.00
7,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99232,"Subsequent hospital inpatient care, typically ...",N,627.0,233,627,72.622456,326.245614,56.366730,57.047895,204556.000003,35341.94
8,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99233,"Subsequent hospital inpatient care, typically ...",N,207.0,127,207,106.653913,461.072464,82.820676,83.196039,95442.000000,17143.88
10,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99291,Critical care delivery critically ill or injur...,N,36.0,23,36,224.170000,1400.000000,175.750000,177.656667,50400.000000,6327.00
39,1003000142,KHALIL,RASHID,NaN,M.D.,M,I,4126 N HOLLAND SYLVANIA RD,SUITE 220,TOLEDO,436233536,OH,US,Anesthesiology,Y,O,99213,Established patient office or other outpatient...,N,129.0,89,129,70.930000,109.000000,50.868062,53.268372,14061.000000,6561.98


In [25]:
df.hcpcs_code.nunique()

10

In [26]:
df.columns

Index(['npi', 'nppes_provider_last_org_name', 'nppes_provider_first_name',
       'nppes_provider_mi', 'nppes_credentials', 'nppes_provider_gender',
       'nppes_entity_code', 'nppes_provider_street1', 'nppes_provider_street2',
       'nppes_provider_city', 'nppes_provider_zip', 'nppes_provider_state',
       'nppes_provider_country', 'provider_type',
       'medicare_participation_indicator', 'place_of_service', 'hcpcs_code',
       'hcpcs_description', 'hcpcs_drug_indicator', 'line_srvc_cnt',
       'bene_unique_cnt', 'bene_day_srvc_cnt', 'average_Medicare_allowed_amt',
       'average_submitted_chrg_amt', 'average_Medicare_payment_amt',
       'average_Medicare_standard_amt', 'Total_submitted_chrg_amt',
       'Total_Medicare_payment_amt'],
      dtype='object')

In [27]:
# need to drop this!
((df.nppes_provider_country != 'US') | (df.medicare_participation_indicator == 'N') |
 (df.nppes_entity_code=='O')).sum()

9458

In [28]:
df['nppes_provider_zip'] = df.nppes_provider_zip.astype(str)
df['nppes_provider_zip'] = df['nppes_provider_zip'].str[0:5]

In [29]:
df = df.loc[(df.nppes_provider_country == 'US') & (df.medicare_participation_indicator == 'Y')\
            & (df.nppes_entity_code=='I')]
dropcols = ["nppes_provider_country", "medicare_participation_indicator", \
            "nppes_entity_code", "nppes_provider_street2", \
            "nppes_provider_mi", "hcpcs_drug_indicator"]

In [30]:
df = df.drop(dropcols, axis=1)

In [31]:
df.columns

Index(['npi', 'nppes_provider_last_org_name', 'nppes_provider_first_name',
       'nppes_credentials', 'nppes_provider_gender', 'nppes_provider_street1',
       'nppes_provider_city', 'nppes_provider_zip', 'nppes_provider_state',
       'provider_type', 'place_of_service', 'hcpcs_code', 'hcpcs_description',
       'line_srvc_cnt', 'bene_unique_cnt', 'bene_day_srvc_cnt',
       'average_Medicare_allowed_amt', 'average_submitted_chrg_amt',
       'average_Medicare_payment_amt', 'average_Medicare_standard_amt',
       'Total_submitted_chrg_amt', 'Total_Medicare_payment_amt'],
      dtype='object')

In [32]:
df.columns = ["npi", "lastname", "firstname", "credentials", "gender", "address", "city",\
              "zip", "state", "provider_type", "place_of_service", "hcpcs_code", "hcpcs_description", \
              "line_srvc_cnt", "bene_unique_cnt", "bene_day_srvc_cnt", \
              "average_Medicare_allowed_amt", "average_submitted_chrg_amt", \
              "average_Medicare_payment_amt", "average_Medicare_standard_amt",\
              "Total_submitted_chrg_amt", "Total_Medicare_payment_amt"]

In [33]:
df.hcpcs_code.value_counts()

99213    457650
99214    428097
99232    175317
99223    133359
99233    120656
99291     62675
99284     60796
99285     57211
66984     15761
A0427         5
Name: hcpcs_code, dtype: int64

In [34]:
df.shape

(1511527, 22)

In [35]:
df.head()

,npi,lastname,firstname,credentials,gender,address,city,zip,state,provider_type,place_of_service,hcpcs_code,hcpcs_description,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,Total_Medicare_payment_amt
6,1003000126,ENKESHAFI,ARDALAN,M.D.,M,900 SETON DR,CUMBERLAND,21502,MD,Internal Medicine,F,99223,"Initial hospital inpatient care, typically 70 ...",114.0,110,114,206.061579,923.991228,160.377193,160.026316,105335.000000,18283.00
7,1003000126,ENKESHAFI,ARDALAN,M.D.,M,900 SETON DR,CUMBERLAND,21502,MD,Internal Medicine,F,99232,"Subsequent hospital inpatient care, typically ...",627.0,233,627,72.622456,326.245614,56.366730,57.047895,204556.000003,35341.94
8,1003000126,ENKESHAFI,ARDALAN,M.D.,M,900 SETON DR,CUMBERLAND,21502,MD,Internal Medicine,F,99233,"Subsequent hospital inpatient care, typically ...",207.0,127,207,106.653913,461.072464,82.820676,83.196039,95442.000000,17143.88
10,1003000126,ENKESHAFI,ARDALAN,M.D.,M,900 SETON DR,CUMBERLAND,21502,MD,Internal Medicine,F,99291,Critical care delivery critically ill or injur...,36.0,23,36,224.170000,1400.000000,175.750000,177.656667,50400.000000,6327.00
39,1003000142,KHALIL,RASHID,M.D.,M,4126 N HOLLAND SYLVANIA RD,TOLEDO,43623,OH,Anesthesiology,O,99213,Established patient office or other outpatient...,129.0,89,129,70.930000,109.000000,50.868062,53.268372,14061.000000,6561.98


In [36]:
LEIE.head()

,LASTNAME,FIRSTNAME,MIDNAME,BUSNAME,GENERAL,SPECIALTY,UPIN,NPI,DOB,ADDRESS,CITY,STATE,ZIP,EXCLTYPE,EXCLDATE,REINDATE,WAIVERDATE,WVRSTATE
0,NaN,NaN,,14 LAWRENCE AVE PHARMACY,PHARMACY,NaN,NaN,0,NaN,14 LAWRENCE AVENUE,SMITHTOWN,NY,11787,1128a1,19880830,0,0,NaN
1,NaN,NaN,,143 MEDICAL EQUIPMENT CO,DME COMPANY,DME - OXYGEN,NaN,0,NaN,701 NW 36 AVENUE,MIAMI,FL,33125,1128b7,19970620,0,0,NaN
2,NaN,NaN,,184TH STREET PHARMACY CORP,OTHER BUSINESS,PHARMACY,NaN,1922348218,NaN,69 E 184TH ST,BRONX,NY,10468,1128a1,20180419,0,0,NaN
3,NaN,NaN,,1951 FLATBUSH AVENUE PHARMACY,PHARMACY,NaN,NaN,0,NaN,1951 FLATBUSH AVE,BROOKLYN,NY,11234,1128b5,20090319,0,0,NaN
4,NaN,NaN,,"1ST COMMUNITY HEALTH CTR, LTD",CLINIC,NaN,NaN,0,NaN,3138 W CERMAK ROAD,CHICAGO,IL,60623,1128a1,19940524,0,0,NaN


In [37]:
LEIE.columns = LEIE.columns.str.lower()

In [38]:
(LEIE.busname.notnull()).sum()

3108

In [39]:
LEIE_indiv = LEIE[LEIE.busname.isnull()]
LEIE_indiv.shape

(69368, 18)

## Matching npi

In [40]:
x = LEIE[LEIE['npi']!=0]
x[x.npi.duplicated()].shape #101 duplicated npis

(101, 18)

In [41]:
LEIE[LEIE.npi!=0].shape[0] #5269 npis not zero

5269

In [42]:
LEIE.npi.nunique() #5169 unique npi

5169

In [43]:
#drop all duplicated npi
LEIE_uniquenpi = LEIE.drop_duplicates(subset='npi')
LEIE_uniquenpi.shape

(5169, 18)

In [44]:
npi_same = pd.merge(df, LEIE_uniquenpi, how='inner',on='npi').npi.values

In [45]:
len(npi_same)

277

In [46]:
df.npi.isin(npi_same).value_counts() #add label column

False    1511250
True         277
Name: npi, dtype: int64

In [47]:
#only 277 npi's matched!
df[df.npi.isin(npi_same)].shape

(277, 22)

In [48]:
df[df.npi.isin(npi_same)]['npi'].nunique()

137

In [49]:
(df.npi.isin(npi_same).astype(int)==1).sum()

277

In [50]:
#df['Target'] = df.npi.isin(npi_same).astype(int)

In [51]:
df.head()

,npi,lastname,firstname,credentials,gender,address,city,zip,state,provider_type,place_of_service,hcpcs_code,hcpcs_description,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,Total_Medicare_payment_amt
6,1003000126,ENKESHAFI,ARDALAN,M.D.,M,900 SETON DR,CUMBERLAND,21502,MD,Internal Medicine,F,99223,"Initial hospital inpatient care, typically 70 ...",114.0,110,114,206.061579,923.991228,160.377193,160.026316,105335.000000,18283.00
7,1003000126,ENKESHAFI,ARDALAN,M.D.,M,900 SETON DR,CUMBERLAND,21502,MD,Internal Medicine,F,99232,"Subsequent hospital inpatient care, typically ...",627.0,233,627,72.622456,326.245614,56.366730,57.047895,204556.000003,35341.94
8,1003000126,ENKESHAFI,ARDALAN,M.D.,M,900 SETON DR,CUMBERLAND,21502,MD,Internal Medicine,F,99233,"Subsequent hospital inpatient care, typically ...",207.0,127,207,106.653913,461.072464,82.820676,83.196039,95442.000000,17143.88
10,1003000126,ENKESHAFI,ARDALAN,M.D.,M,900 SETON DR,CUMBERLAND,21502,MD,Internal Medicine,F,99291,Critical care delivery critically ill or injur...,36.0,23,36,224.170000,1400.000000,175.750000,177.656667,50400.000000,6327.00
39,1003000142,KHALIL,RASHID,M.D.,M,4126 N HOLLAND SYLVANIA RD,TOLEDO,43623,OH,Anesthesiology,O,99213,Established patient office or other outpatient...,129.0,89,129,70.930000,109.000000,50.868062,53.268372,14061.000000,6561.98


## Merging Aparna's Files:

In [52]:
leie_corrected = pd.read_csv('data/Aparna/LEIE_reduced.csv', index_col=0)

In [53]:
CMS_subset = pd.read_csv('data/Aparna/LEIE_labels_1.csv', index_col=0)

In [54]:
CMS_subset.hcpcs_code.nunique()

9

In [55]:
leie_corrected.shape

(64432, 21)

In [56]:
colskeep = ["lastname", "npi_leie", "zip_new", "street1", "EXCLTYPE", "EXCLDATE"]

In [57]:
leie_corrected = leie_corrected.filter(items=colskeep, axis=1)

In [58]:
leie_corrected.shape

(64432, 6)

In [59]:
leie_corrected.head()

,lastname,npi_leie,zip_new,street1,EXCLTYPE,EXCLDATE
3108,aalders,0,60453,9527 south kenton,1128b4,20030820
3109,aaltonen,0,99201,32 w pacific ave,1128a4,20120419
3110,aamir,0,48302,3956 columbia court,1128a1,20170220
3111,aaron,0,85018,3325 n rose circle drive,1128b4,20171019
3112,aaron,0,35504,5911 old pineywoods rd,1128b4,20061120


In [60]:
leie_corrected['year'] = pd.to_datetime(leie_corrected["EXCLDATE"], format="%Y%m%d").dt.year

In [61]:
leie_corrected.head()

,lastname,npi_leie,zip_new,street1,EXCLTYPE,EXCLDATE,year
3108,aalders,0,60453,9527 south kenton,1128b4,20030820,2003
3109,aaltonen,0,99201,32 w pacific ave,1128a4,20120419,2012
3110,aamir,0,48302,3956 columbia court,1128a1,20170220,2017
3111,aaron,0,85018,3325 n rose circle drive,1128b4,20171019,2017
3112,aaron,0,35504,5911 old pineywoods rd,1128b4,20061120,2006


In [62]:
cmscolskeep = ["npi", "lastname", "nppes_provider_first_name", "zip_new","street1"]

In [63]:
CMS_subset = CMS_subset.filter(items=cmscolskeep, axis=1)

In [64]:
CMS_subset.head()

,npi,lastname,nppes_provider_first_name,zip_new,street1
7901,1003044611,danek,SAMANTHA,49201,205 n east ave
7902,1003044611,danek,SAMANTHA,49201,205 n east ave
19615,1003125485,hanna,EHAB,11706,301 e main st
25144,1003184599,eastman,MEGAN,50501,800 kenyon rd
25145,1003184599,eastman,MEGAN,50501,800 kenyon rd


In [65]:
CMS_subset.shape

(3915, 5)

In [66]:
merged_file = pd.merge(CMS_subset, leie_corrected, how="inner", left_on=["lastname", "zip_new", "street1"],\
         right_on=["lastname","zip_new", "street1"])

In [67]:
merged_file #total 8

,npi,lastname,nppes_provider_first_name,zip_new,street1,npi_leie,EXCLTYPE,EXCLDATE,year
0,1285969980,siddiqui,IQBAL,7936,9 tower dr,0,1128a1,20010719,2001
1,1285969980,siddiqui,IQBAL,7936,9 tower dr,0,1128a1,20010719,2001
2,1285969980,siddiqui,IQBAL,7936,9 tower dr,0,1128a1,20010719,2001
3,1447436209,fisher,EMILY,77030,6560 fannin st,0,1128b4,19981020,1998
4,1447436209,fisher,EMILY,77030,6560 fannin st,0,1128b4,19981020,1998
5,1518920602,lee,VICTOR,95123,5710 cahalan ave,0,1128a1,19830107,1983
6,1891090262,antoon,PATRICK,71860,218 church st,0,1128a1,20061120,2006
7,1891090262,antoon,PATRICK,71860,218 church st,0,1128a1,20061120,2006


## Adding EXCLTYPE and EXCLDATE

In [68]:
#LEIE
LEIE = pd.read_csv('data/LEIE_08_2019.csv')

In [69]:
LEIE.columns = LEIE.columns.str.lower()

In [70]:
LEIE_indiv = LEIE[LEIE.busname.isnull()]
LEIE_indiv.shape

(69368, 18)

In [71]:
LEIE_indiv['year'] = pd.to_datetime(LEIE_indiv["excldate"], format="%Y%m%d").dt.year

/Users/rad/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [72]:
LEIE_indiv.head()

,lastname,firstname,midname,busname,general,specialty,upin,npi,dob,address,city,state,zip,excltype,excldate,reindate,waiverdate,wvrstate,year
3108,AALDERS,NANCY,BLOCK,NaN,HOSPITAL,NURSE/NURSES AIDE,NaN,0,19640305.0,9527 SOUTH KENTON,OAK LAWN,IL,60453,1128b4,20030820,0,0,NaN,2003
3109,AALTONEN,NICKOLAS,A,NaN,IND- LIC HC SERV PRO,PHARMACY TECHNICIAN,NaN,0,19880123.0,32 W PACIFIC AVE,SPOKANE,WA,99201,1128a4,20120419,0,0,NaN,2012
3110,AAMIR,MUHAMMAD,SOHAIL,NaN,BUS OWNER/EXEC,HOME HEALTH AGENCY,NaN,0,19700910.0,3956 COLUMBIA COURT,BLOOMFIELD HILLS,MI,48302,1128a1,20170220,0,0,NaN,2017
3111,AARON,ALINA,A,NaN,IND- LIC HC SERV PRO,NURSE/NURSES AIDE,NaN,0,19870209.0,3325 N ROSE CIRCLE DRIVE,PHOENIX,AZ,85018,1128b4,20171019,0,0,NaN,2017
3112,AARON,BARBARA,KAY,NaN,NURSING PROFESSION,NURSE/NURSES AIDE,NaN,0,19650914.0,5911 OLD PINEYWOODS RD,JASPER,AL,35504,1128b4,20061120,0,0,NaN,2006


In [73]:
merged_file

,npi,lastname,nppes_provider_first_name,zip_new,street1,npi_leie,EXCLTYPE,EXCLDATE,year
0,1285969980,siddiqui,IQBAL,7936,9 tower dr,0,1128a1,20010719,2001
1,1285969980,siddiqui,IQBAL,7936,9 tower dr,0,1128a1,20010719,2001
2,1285969980,siddiqui,IQBAL,7936,9 tower dr,0,1128a1,20010719,2001
3,1447436209,fisher,EMILY,77030,6560 fannin st,0,1128b4,19981020,1998
4,1447436209,fisher,EMILY,77030,6560 fannin st,0,1128b4,19981020,1998
5,1518920602,lee,VICTOR,95123,5710 cahalan ave,0,1128a1,19830107,1983
6,1891090262,antoon,PATRICK,71860,218 church st,0,1128a1,20061120,2006
7,1891090262,antoon,PATRICK,71860,218 church st,0,1128a1,20061120,2006


In [74]:
merged_file = merged_file.drop(['lastname', 'nppes_provider_first_name', 'zip_new', 'street1','npi_leie','EXCLDATE'], axis=1)
merged_file

,npi,EXCLTYPE,year
0,1285969980,1128a1,2001
1,1285969980,1128a1,2001
2,1285969980,1128a1,2001
3,1447436209,1128b4,1998
4,1447436209,1128b4,1998
5,1518920602,1128a1,1983
6,1891090262,1128a1,2006
7,1891090262,1128a1,2006


In [75]:
keepcols1 = ['npi', 'excltype', 'year']
LEIE_x = LEIE_indiv[LEIE_indiv.npi!=0].filter(items=keepcols1)
LEIE_x.columns = ['npi', 'EXCLTYPE', 'year']
LEIE_x.head()

,npi,EXCLTYPE,year
3123,1477537496,1128b4,2014
3142,1194807255,1128b4,2018
3143,1225029184,1128a2,2014
3212,1487870507,1128a4,2018
3216,1497811368,1128a1,1999


In [76]:
print(merged_file.shape)
print(LEIE_x.shape)

(8, 3)
(4934, 3)


In [77]:
LEIE_npi_match = pd.concat([merged_file, LEIE_x], sort=False)
LEIE_npi_match.head()

,npi,EXCLTYPE,year
0,1285969980,1128a1,2001
1,1285969980,1128a1,2001
2,1285969980,1128a1,2001
3,1447436209,1128b4,1998
4,1447436209,1128b4,1998


In [78]:
LEIE_npi_match.shape

(4942, 3)

In [79]:
df.shape

(1511527, 22)

In [80]:
combo = pd.merge(df, LEIE_npi_match, how="left", on="npi")
combo.head()

,npi,lastname,firstname,credentials,gender,address,city,zip,state,provider_type,place_of_service,hcpcs_code,hcpcs_description,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,Total_Medicare_payment_amt,EXCLTYPE,year
0,1003000126,ENKESHAFI,ARDALAN,M.D.,M,900 SETON DR,CUMBERLAND,21502,MD,Internal Medicine,F,99223,"Initial hospital inpatient care, typically 70 ...",114.0,110,114,206.061579,923.991228,160.377193,160.026316,105335.000000,18283.00,NaN,NaN
1,1003000126,ENKESHAFI,ARDALAN,M.D.,M,900 SETON DR,CUMBERLAND,21502,MD,Internal Medicine,F,99232,"Subsequent hospital inpatient care, typically ...",627.0,233,627,72.622456,326.245614,56.366730,57.047895,204556.000003,35341.94,NaN,NaN
2,1003000126,ENKESHAFI,ARDALAN,M.D.,M,900 SETON DR,CUMBERLAND,21502,MD,Internal Medicine,F,99233,"Subsequent hospital inpatient care, typically ...",207.0,127,207,106.653913,461.072464,82.820676,83.196039,95442.000000,17143.88,NaN,NaN
3,1003000126,ENKESHAFI,ARDALAN,M.D.,M,900 SETON DR,CUMBERLAND,21502,MD,Internal Medicine,F,99291,Critical care delivery critically ill or injur...,36.0,23,36,224.170000,1400.000000,175.750000,177.656667,50400.000000,6327.00,NaN,NaN
4,1003000142,KHALIL,RASHID,M.D.,M,4126 N HOLLAND SYLVANIA RD,TOLEDO,43623,OH,Anesthesiology,O,99213,Established patient office or other outpatient...,129.0,89,129,70.930000,109.000000,50.868062,53.268372,14061.000000,6561.98,NaN,NaN


In [81]:
combo.shape

(1511539, 24)

In [82]:
combo.EXCLTYPE.notnull().sum()

297

In [83]:
def imputeNone(df_input):
    df = df_input.copy()
    cols = ['EXCLTYPE', 'year']
    for col in cols:
        df[col] = np.where(df[col].isnull(), "None", df[col])
    return(df)

In [84]:
combo = imputeNone(combo)

In [85]:
combo.head()

,npi,lastname,firstname,credentials,gender,address,city,zip,state,provider_type,place_of_service,hcpcs_code,hcpcs_description,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,Total_Medicare_payment_amt,EXCLTYPE,year
0,1003000126,ENKESHAFI,ARDALAN,M.D.,M,900 SETON DR,CUMBERLAND,21502,MD,Internal Medicine,F,99223,"Initial hospital inpatient care, typically 70 ...",114.0,110,114,206.061579,923.991228,160.377193,160.026316,105335.000000,18283.00,None,None
1,1003000126,ENKESHAFI,ARDALAN,M.D.,M,900 SETON DR,CUMBERLAND,21502,MD,Internal Medicine,F,99232,"Subsequent hospital inpatient care, typically ...",627.0,233,627,72.622456,326.245614,56.366730,57.047895,204556.000003,35341.94,None,None
2,1003000126,ENKESHAFI,ARDALAN,M.D.,M,900 SETON DR,CUMBERLAND,21502,MD,Internal Medicine,F,99233,"Subsequent hospital inpatient care, typically ...",207.0,127,207,106.653913,461.072464,82.820676,83.196039,95442.000000,17143.88,None,None
3,1003000126,ENKESHAFI,ARDALAN,M.D.,M,900 SETON DR,CUMBERLAND,21502,MD,Internal Medicine,F,99291,Critical care delivery critically ill or injur...,36.0,23,36,224.170000,1400.000000,175.750000,177.656667,50400.000000,6327.00,None,None
4,1003000142,KHALIL,RASHID,M.D.,M,4126 N HOLLAND SYLVANIA RD,TOLEDO,43623,OH,Anesthesiology,O,99213,Established patient office or other outpatient...,129.0,89,129,70.930000,109.000000,50.868062,53.268372,14061.000000,6561.98,None,None


In [86]:
combo.shape

(1511539, 24)

## Adding Target Column

In [87]:
len(npi_same)

277

In [88]:
npi_same_list = npi_same.tolist()

In [ ]:
npi_more = merged_file.npi.tolist()
npi_more

[1285969980,
 1285969980,
 1285969980,
 1447436209,
 1447436209,
 1518920602,
 1891090262,
 1891090262]

In [ ]:
for i in npi_more:
    npi_same_list.append(i)

In [ ]:
len(npi_same_list)

285

In [ ]:
(combo.npi.isin(npi_same_list)).sum()

297

In [ ]:
combo['Target'] = (combo.npi.isin(npi_same_list)).astype(int)

In [ ]:
(combo['Target']==1).sum()

297

In [ ]:
combo[combo.Target==1]['npi'].nunique()

141

In [ ]:
combo.head()

,npi,lastname,firstname,credentials,gender,address,city,zip,state,provider_type,place_of_service,hcpcs_code,hcpcs_description,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,Total_Medicare_payment_amt,EXCLTYPE,year,Target
0,1003000126,ENKESHAFI,ARDALAN,M.D.,M,900 SETON DR,CUMBERLAND,21502,MD,Internal Medicine,F,99223,"Initial hospital inpatient care, typically 70 ...",114.0,110,114,206.061579,923.991228,160.377193,160.026316,105335.000000,18283.00,None,None,0
1,1003000126,ENKESHAFI,ARDALAN,M.D.,M,900 SETON DR,CUMBERLAND,21502,MD,Internal Medicine,F,99232,"Subsequent hospital inpatient care, typically ...",627.0,233,627,72.622456,326.245614,56.366730,57.047895,204556.000003,35341.94,None,None,0
2,1003000126,ENKESHAFI,ARDALAN,M.D.,M,900 SETON DR,CUMBERLAND,21502,MD,Internal Medicine,F,99233,"Subsequent hospital inpatient care, typically ...",207.0,127,207,106.653913,461.072464,82.820676,83.196039,95442.000000,17143.88,None,None,0
3,1003000126,ENKESHAFI,ARDALAN,M.D.,M,900 SETON DR,CUMBERLAND,21502,MD,Internal Medicine,F,99291,Critical care delivery critically ill or injur...,36.0,23,36,224.170000,1400.000000,175.750000,177.656667,50400.000000,6327.00,None,None,0
4,1003000142,KHALIL,RASHID,M.D.,M,4126 N HOLLAND SYLVANIA RD,TOLEDO,43623,OH,Anesthesiology,O,99213,Established patient office or other outpatient...,129.0,89,129,70.930000,109.000000,50.868062,53.268372,14061.000000,6561.98,None,None,0


In [ ]:
combo.dtypes

npi                                int64
lastname                          object
firstname                         object
credentials                       object
gender                            object
address                           object
city                              object
zip                               object
state                             object
provider_type                     object
place_of_service                  object
hcpcs_code                        object
hcpcs_description                 object
line_srvc_cnt                    float64
bene_unique_cnt                    int64
bene_day_srvc_cnt                  int64
average_Medicare_allowed_amt     float64
average_submitted_chrg_amt       float64
average_Medicare_payment_amt     float64
average_Medicare_standard_amt    float64
Total_submitted_chrg_amt         float64
Total_Medicare_payment_amt       float64
EXCLTYPE                          object
year                              object
Target          

In [ ]:
cols = ['lastname', 'firstname', 'address', 'city']
for col in cols:
    combo[col] = combo[col].str.lower()

In [ ]:
combo.head()

,npi,lastname,firstname,credentials,gender,address,city,zip,state,provider_type,place_of_service,hcpcs_code,hcpcs_description,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,Total_Medicare_payment_amt,EXCLTYPE,year,Target
0,1003000126,enkeshafi,ardalan,M.D.,M,900 seton dr,cumberland,21502,MD,Internal Medicine,F,99223,"Initial hospital inpatient care, typically 70 ...",114.0,110,114,206.061579,923.991228,160.377193,160.026316,105335.000000,18283.00,None,None,0
1,1003000126,enkeshafi,ardalan,M.D.,M,900 seton dr,cumberland,21502,MD,Internal Medicine,F,99232,"Subsequent hospital inpatient care, typically ...",627.0,233,627,72.622456,326.245614,56.366730,57.047895,204556.000003,35341.94,None,None,0
2,1003000126,enkeshafi,ardalan,M.D.,M,900 seton dr,cumberland,21502,MD,Internal Medicine,F,99233,"Subsequent hospital inpatient care, typically ...",207.0,127,207,106.653913,461.072464,82.820676,83.196039,95442.000000,17143.88,None,None,0
3,1003000126,enkeshafi,ardalan,M.D.,M,900 seton dr,cumberland,21502,MD,Internal Medicine,F,99291,Critical care delivery critically ill or injur...,36.0,23,36,224.170000,1400.000000,175.750000,177.656667,50400.000000,6327.00,None,None,0
4,1003000142,khalil,rashid,M.D.,M,4126 n holland sylvania rd,toledo,43623,OH,Anesthesiology,O,99213,Established patient office or other outpatient...,129.0,89,129,70.930000,109.000000,50.868062,53.268372,14061.000000,6561.98,None,None,0


In [ ]:
combo.shape #df.shape = (1511527, 22)

(1511539, 25)

In [ ]:
combo.head()

,npi,lastname,firstname,credentials,gender,address,city,zip,state,provider_type,place_of_service,hcpcs_code,hcpcs_description,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,Total_Medicare_payment_amt,EXCLTYPE,year,Target
0,1003000126,enkeshafi,ardalan,M.D.,M,900 seton dr,cumberland,21502,MD,Internal Medicine,F,99223,"Initial hospital inpatient care, typically 70 ...",114.0,110,114,206.061579,923.991228,160.377193,160.026316,105335.000000,18283.00,None,None,0
1,1003000126,enkeshafi,ardalan,M.D.,M,900 seton dr,cumberland,21502,MD,Internal Medicine,F,99232,"Subsequent hospital inpatient care, typically ...",627.0,233,627,72.622456,326.245614,56.366730,57.047895,204556.000003,35341.94,None,None,0
2,1003000126,enkeshafi,ardalan,M.D.,M,900 seton dr,cumberland,21502,MD,Internal Medicine,F,99233,"Subsequent hospital inpatient care, typically ...",207.0,127,207,106.653913,461.072464,82.820676,83.196039,95442.000000,17143.88,None,None,0
3,1003000126,enkeshafi,ardalan,M.D.,M,900 seton dr,cumberland,21502,MD,Internal Medicine,F,99291,Critical care delivery critically ill or injur...,36.0,23,36,224.170000,1400.000000,175.750000,177.656667,50400.000000,6327.00,None,None,0
4,1003000142,khalil,rashid,M.D.,M,4126 n holland sylvania rd,toledo,43623,OH,Anesthesiology,O,99213,Established patient office or other outpatient...,129.0,89,129,70.930000,109.000000,50.868062,53.268372,14061.000000,6561.98,None,None,0


In [ ]:
combo.drop_duplicates().sum()

In [ ]:
combo_1 = combo.drop_duplicates()

In [ ]:
combo_1.to_csv("data/combo.csv",index_label=False) #write to csv

In [ ]:
combo_1.shape

In [ ]:
combo_1.hcpcs_code.value_counts()

In [ ]:
combo_1.hcpcs_code.nunique()

In [ ]:
dfx= pd.read_csv("data/combo.csv")

In [ ]:
dfx.shape

In [ ]:
dfx.hcpcs_code.nunique()

In [ ]:
for i in dfx.hcpcs_code.unique():
    print(type(i))

In [ ]:
dfx.hcpcs_code.value_counts()